In [2]:
import torch
from torch import nn
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

# 填充和步幅

## 填充

### 为什么有填充的问题？

- 更大的卷积核可以更快地减小输出大小
  - 形状从输入的$n_h × n_w$,减少到输出的$(n_h一k_h+ 1)×(n_w - k_w + 1)$

\begin{example}\label{example:reducedSize}
给定(32$\times$ 32)输入图像，应用5$\times$5大小的卷积核
    \begin{itemize}
    \item 第1层得到输出大小28$\times$28
    \item 第7层得到输出大小4$\times$4（因此层数不能超过7层）
    \end{itemize}
\end{example}


- 如何解决网络深度受到限制？
    - 在**输入周围添加额外的行/列**

\begin{definition}\label{def:padding}
**填充（padding）**：在输入图像的边界填充元素（通常填充元素是$0$）。
\end{definition}

\begin{example}\label{example:padding}
将$3 \times 3$输入填充到$5 \times 5$，那么它的输出就增加为$4 \times 4$
\end{example}


![](padding0.png)

从这个例子中可以发现，如果没有填充，输出的形状为2\*2，但是现在的输出是4\*4，比输入（3*3）还大。

### 填充的方法

- 对输入填充$p_h$行（大约一半在顶部，一半在底部）和$p_w$列（左侧大约一半，右侧一半），输出形状为
$$(n_h-k_h+p_h+1)\times (n_w-k_w+p_w+1)$$


- 通常取$p_h = k_h-1$, $p_w=k_w-1$，以使得输出的形状和输入的形状保持一致

- 当$k_h$为偶数：在上下两侧填充$p_h/2$
- 当$k_h$为奇数：在上侧填充$\lceil{p_h/2}\rceil$，在下侧填充$\lfloor{p_h/2}\rfloor$

- 卷积神经网络中**卷积核的高度和宽度通常为奇数**
    - 例如1、3、5或7

- 使用奇数的核大小和填充大小提供了书写上的便利。对于任何二维张量`X`，当满足：
    1. 卷积核的大小是奇数；
    2. 所有边的填充行数和列数相同；
    3. 输出与输入具有相同高度和宽度

则可以得出：输出`Y[i, j]`是通过以输入`X[i, j]`为中心，与卷积核进行互相关计算得到的

\begin{example}\label{example:paddingExample}
在所有侧边填充1个像素的情形。当核为3$\times$3，padding=1（上下各填充一行，左右各填充一行，即$p_h=2, p_w=2$）时，输出与输入的形状一致。
\end{example}

In [3]:
# 为了方便起见，定义一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

In [4]:
# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8)) # 生成8*8的随机张量
X
comp_conv2d(conv2d,X)
comp_conv2d(conv2d, X).shape

tensor([[0.7175, 0.0345, 0.6503, 0.1917, 0.7842, 0.8561, 0.7978, 0.3539],
        [0.2177, 0.0756, 0.2116, 0.4202, 0.6780, 0.1298, 0.5513, 0.6331],
        [0.2281, 0.3408, 0.7426, 0.6783, 0.8877, 0.6139, 0.3896, 0.8620],
        [0.7513, 0.8154, 0.9085, 0.1218, 0.2803, 0.2553, 0.6905, 0.8001],
        [0.6058, 0.6628, 0.2646, 0.1189, 0.5386, 0.9695, 0.8801, 0.5995],
        [0.5972, 0.7698, 0.9739, 0.4841, 0.2236, 0.6200, 0.8905, 0.7847],
        [0.7357, 0.6533, 0.6967, 0.8520, 0.1354, 0.1894, 0.4211, 0.3424],
        [0.5817, 0.1980, 0.1671, 0.1179, 0.6421, 0.2377, 0.4558, 0.1321]])

tensor([[-0.1509, -0.0978, -0.1962, -0.2682, -0.3667, -0.1798, -0.2400, -0.0960],
        [ 0.0866,  0.0292, -0.0948, -0.0745,  0.1318,  0.3333,  0.1494,  0.0373],
        [-0.2944, -0.3126, -0.2014,  0.1323,  0.1062,  0.0842, -0.0701,  0.0930],
        [-0.3087, -0.0444,  0.3493,  0.4263,  0.1925, -0.0282, -0.0429,  0.2430],
        [-0.0341,  0.2295,  0.2066,  0.0369, -0.1561, -0.1731,  0.0532,  0.2580],
        [-0.1477, -0.0092, -0.0948, -0.0134,  0.2462,  0.2643,  0.2597,  0.2943],
        [-0.0402,  0.3610,  0.3297,  0.2371,  0.0885,  0.2095,  0.3033,  0.3839],
        [ 0.2380,  0.4288,  0.4207,  0.2767,  0.1050,  0.0913,  0.1604,  0.1833]],
       grad_fn=<ReshapeAliasBackward0>)

torch.Size([8, 8])

```python
torch.nn.Conv2d(in_channels, out_channels, kernel_size, padding=0, padding_mode='zeros')
```

- `padding`：整数、tuple或者字符串，整数表明高和宽增加相同的填充；tuple包含两个元素，分别指明宽和高的填充；如果设定为'same'，表明输出的形状与输入一致

- 当卷积核的高度和宽度不同时，可以[**填充不同的高度和宽度**]，使输出和输入具有相同的高度和宽度

\begin{example}\label{example:paddingDiferentSize}
使用高度为5，宽度为3的卷积核，高度和宽度两边的填充分别为2和1。
\end{example}


In [5]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

## 步幅

- 如果说填充是防止小图片的形状随深度的增加而快速变小，那么步幅是**加快大图片的形状变小**

\begin{problem}\label{prob:whyStride}
为什么需要步幅？
\end{problem}

\begin{example}\label{example:whyStride}
给定输入大小224$\times$224，在使用5$\times$5卷积核的情况下，需要55层将输出降低到4$\times$4
\end{example}

需要大量计算才能得到较小输出

- 在计算互相关时，卷积窗口从输入张量的左上角开始，向下、向右滑动。
- 前面的例子默认每次滑动一个元素。
- 为了高效计算或是缩减采样次数，卷积窗口可以跳过中间位置，每次滑动多个元素

\begin{definition}\label{def:stride}
步幅（stride）：行或列每次滑动元素的数量。
\end{definition}

\begin{example}\label{example:stride}
垂直步幅为3，水平步幅为2
\end{example}


  
  <img src='stride0.png' class='float_right h-80'>

- 给定垂直步幅$s_h$和水平步幅$s_w$，输出形状是
$$\lfloor (n_h-k_h+p_h+s_h)/s_h \rfloor \times \lfloor (n_w-k_w+p_w+s_w)/s_w \rfloor$$

- 如果$p_h = k_h-1$, $p_w = k_w-1$
$$\lfloor (n_h+s_h-1)/s_h \rfloor \times \lfloor (n_w+s_w-1)/s_w \rfloor$$

- 如果输入高度和宽度可以被步幅整除
$$(n_h/s_h)\times (n_w/s_w)$$

```python
torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
```

- `stride`：整数或者tuple，整数表明高和宽具有相同的步幅；tuple有两个元素，分别表明高和宽的步幅

\begin{example}\label{example:strid2}
将高度和宽度的步幅设置为2，从而将输入的高度和宽度减半。
\end{example}


In [6]:
conv2d  =  nn.Conv2d(1,1,kernel_size=3, padding=1,stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

\begin{example}\label{example:stride3}
将高度和宽度的步幅分别设置为3和4。
\end{example}


In [9]:
conv2d = nn.Conv2d(1,1,kernel_size=(5,3),padding=(1,0),stride=(4,3))

<center><img src="../img/6_convolutional_neural_networks/convolutionPaddingStrides.gif" width=60%></center>

In [8]:
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

## 总结

- **填充和步幅是卷积层的超参数**(上节课还有一个超参数，是**卷积核的大小**)
- 填充在输入周围添加额外的行/列，来控制输出形状的减少量
- 步幅是每次滑动核窗口时的行/列的步长，可以成倍的减少输出形状

- 当输入高度和宽度两侧的填充数量分别为$p_h$和$p_w$时，称之为填充$(p_h, p_w)$
- 当$p_h = p_w = p$时，填充是$p$

- 当高度和宽度上的步幅分别为$s_h$和$s_w$时，称之为步幅$(s_h, s_w)$
- 特别地，当$s_h = s_w = s$时，称步幅为$s$

- 默认情况下，填充为0，步幅为1
- 在实践中，很少使用不一致的步幅或填充，也就是说，通常有$p_h = p_w$和$s_h = s_w$